In [ ]:
#package import statements
import random
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import math
import seaborn as sns
import pandas as pd
from tqdm import tqdm

In [ ]:
#Function for belief agggregation and updation
def next_period_beliefs(number_of_agents,beliefs,relationship_matrix,weight_matrix):
    weight_matrix_star=np.sum(np.multiply(weight_matrix,relationship_matrix),axis = 1)# axis = 1 means horizontal summation
    weight_matrix_star_resizer=np.reshape(np.repeat(weight_matrix_star,number_of_agents,axis=0),(number_of_agents,number_of_agents))
    weight_matrix_star_restricted=np.divide(np.multiply(weight_matrix,relationship_matrix),weight_matrix_star_resizer)
    return np.matmul(weight_matrix_star_restricted,beliefs)

In [ ]:
#Function to find the immediate neighbours of an agent
def neighbours(agent):
    nei=[]
    for i in range(len(agent)):
        if agent[i] == 1:
            nei.append(i)
    return nei   

In [ ]:
#Function defining the neighbourhood change process
def neighbour_change(number_of_agents,relationship_matrix,beliefs,ub=0.1,lb=0.1):
    for agent in range(number_of_agents):
        # key agent has connections or is neighbours with nei
        nei = neighbours(relationship_matrix[agent])
        diff=[]
        for i in nei:
            diff.append(abs(beliefs[agent]- beliefs[i]))
        nei_be = beliefs[[i for i,x in enumerate(diff) if x == max(diff)][0]]
        if nei_be <= min(1,beliefs[agent]+ub) and nei_be >= max(0,beliefs[agent]-lb):
            replaced_neighbour = neighbours(relationship_matrix[agent])[[i for i,x in enumerate(diff) if x == max(diff)][0]]
            nei.remove(replaced_neighbour)# this removes the neighbour with the max difference in belief given that it is outside the bounds
            nei_nei = []
            for i in nei:
                neigh = neighbours(relationship_matrix[i])
                for j in neigh:
                    nei_nei.append(j)
            nei_nei= list(set(nei_nei))
            for i in nei:
                if i in nei_nei:
                    nei_nei.remove(i)
            simi = []
            for i in nei_nei:
                simi.append(abs(beliefs[agent]- beliefs[i]))
            if len(simi)>0:
                added_neighbour = nei_nei[[i for i,x in enumerate(simi) if x == min(simi)][0]]
                relationship_matrix[agent][replaced_neighbour] = 0
                relationship_matrix[replaced_neighbour][agent] = 0
                relationship_matrix[agent][added_neighbour] = 1
                relationship_matrix[added_neighbour][agent] = 1
    return relationship_matrix

In [ ]:
def plot_graphs(time_periods,consensus,number_of_agents):
    for i in range(len(consensus)):
        plt.plot(np.linspace(0,time_periods,time_periods+1),list(consensus.iloc[i]),lw= 1)
        plt.xlabel("Time")
        plt.ylabel("Belief values")
        plt.title(str(number_of_agents)+" Agents")

#Complete Graph Model

In [ ]:
def siml(number_of_agents,time_periods):
    #Network = nx.Graph()
    # randomly generating initial belief values i.e. x(t) for the agents and storing them in the node 
    '''
    beliefs = []
    for i in range(number_of_agents):
        belf = np.random.rand(1)[0]
        #Network.add_node(i,belief = belf)
        beliefs.append(belf)
    '''
    beliefs=np.random.uniform(0,1,number_of_agents)
    relationship_matrix= np.where(np.random.rand(number_of_agents,number_of_agents) + np.identity(number_of_agents) > 0.0, 1, 0)
    weight_matrix = np.random.rand(number_of_agents,number_of_agents)
    
    consensus = pd.DataFrame({'t0':beliefs})
    b=beliefs
    for i in range(time_periods):
        b=next_period_beliefs(number_of_agents,b, relationship_matrix, weight_matrix)
        consensus.insert(loc= len(consensus.columns),column = ('t'+str(i+1)),value= b)
    return consensus

In [ ]:
def auto_siml(number_of_agents):
    #Network = nx.Graph()
    # randomly generating initial belief values i.e. x(t) for the agents and storing them in the node 
    '''
    beliefs = []
    for i in range(number_of_agents):
        belf = np.random.rand(1)[0]
        #Network.add_node(i,belief = belf)
        beliefs.append(belf)
    '''
    beliefs=np.random.uniform(0,1,number_of_agents)
    relationship_matrix= np.ones((number_of_agents,number_of_agents))
    weight_matrix = np.random.rand(number_of_agents,number_of_agents)
    
    consensus = pd.DataFrame({'t0':beliefs})
    b=beliefs
    t=0
    consensus_final = list(consensus[consensus.columns[-1]])
    dif_sum=[]
    for j in consensus_final:
        for k in consensus_final:
            dif_sum.append(abs(j-k))
    while(sum(dif_sum)/len(dif_sum) > 0.001):
        t=t+1
        if t > 99:
            break
        b=next_period_beliefs(number_of_agents,b, relationship_matrix, weight_matrix)
        consensus.insert(loc= len(consensus.columns),column = ('t'+str(t)),value= b)
        consensus_final = list(consensus[consensus.columns[-1]])
        dif_sum=[]
        for j in consensus_final:
            for k in consensus_final:
                dif_sum.append(abs(j-k))
    return t

In [ ]:
complete_graph_sim =pd.DataFrame()
for s in tqdm(range(101)):
    time_required=[]
    for i in range(2,102):
        time_required.append(auto_siml(i))
    complete_graph_sim.insert(loc= len(complete_graph_sim.columns),column = "sim_"+str(s),value = time_required)

In [ ]:
mean=complete_graph_sim.mean(axis=1)
complete_graph_sim.insert(loc=len(complete_graph_sim.columns),column = "number_of_agents",value = np.linspace(2,101,100) )
complete_graph_sim.insert(loc=len(complete_graph_sim.columns),column = "Mean",value =mean )
complete_graph_sim.to_csv('complete_graph_sim.csv') # to save the results to a csv file.

#Restricted Graph Model

In [ ]:
def siml(number_of_agents,time_periods):
    #Network = nx.Graph()
    # randomly generating initial belief values i.e. x(t) for the agents and storing them in the node 
    sparseness=0.5
    beliefs=np.random.uniform(0,1,number_of_agents)
    a = np.random.rand(number_of_agents,number_of_agents)
    relationship_matrix = np.where((np.tril(a) + np.tril(a, -1).T)+np.eye(number_of_agents)>sparseness,1,0)
    weight_matrix = np.random.rand(number_of_agents,number_of_agents)
    
    consensus = pd.DataFrame({'t0':beliefs})
    b=beliefs
    for i in range(time_periods):
        b=next_period_beliefs(number_of_agents,b, relationship_matrix, weight_matrix)
        consensus.insert(loc= len(consensus.columns),column = ('t'+str(i+1)),value= b)
    return consensus

In [ ]:
def auto_siml(number_of_agents):
    #Network = nx.Graph()
    # randomly generating initial belief values i.e. x(t) for the agents and storing them in the node 
    #print('3')
    sparseness=0.75
    beliefs=np.random.uniform(0,1,number_of_agents)
    a = np.random.rand(number_of_agents,number_of_agents)
    relationship_matrix = np.where((np.tril(a) + np.tril(a, -1).T)+np.eye(number_of_agents)>sparseness,1,0)
    weight_matrix = np.random.rand(number_of_agents,number_of_agents)

    consensus = pd.DataFrame({'t0':beliefs})
    b=beliefs.copy()
    t=0
    consensus_final = list(consensus[consensus.columns[-1]])
    dif_sum=[]
    for j in consensus_final:
        for k in consensus_final:
            dif_sum.append(abs(j-k))
    while(sum(dif_sum)/len(dif_sum) > 0.001):
        t=t+1
        if t>99:
            break
        b=next_period_beliefs(number_of_agents,b, relationship_matrix, weight_matrix)
        consensus.insert(loc= len(consensus.columns),column = ('t'+str(t)),value= b)
        consensus_final = list(consensus[consensus.columns[-1]])
        dif_sum=[]
        for j in consensus_final:
            for k in consensus_final:
                dif_sum.append(abs(j-k))
    return t

In [ ]:
restricted_graph_sim =pd.DataFrame()
for s in tqdm(range(101)):
    #print('1')
    time_required=[]
    for i in range(2,101):
        #print('2')
        time_required.append(auto_siml(i))
    restricted_graph_sim.insert(loc= len(restricted_graph_sim.columns),column = "sim_"+str(s),value = time_required)

In [ ]:
mean=restricted_graph_sim.mean(axis=1)
restricted_graph_sim.insert(loc=len(restricted_graph_sim.columns),column = "number_of_agents",value = np.linspace(2,101,100) )
restricted_graph_sim.insert(loc=len(restricted_graph_sim.columns),column = "Mean",value =mean )
restricted_graph_sim.to_csv('restricted_graph_302_s=0.75.csv')

#Restricted Graph Model with Neighbour Change

In [ ]:
def siml(number_of_agents,time_periods):
    #Network = nx.Graph()
    # randomly generating initial belief values i.e. x(t) for the agents and storing them in the node 
    sparseness=0.5
    beliefs=np.random.uniform(0,1,number_of_agents)
    a = np.random.rand(number_of_agents,number_of_agents)
    relationship_matrix = np.where((np.tril(a) + np.tril(a, -1).T)+np.eye(number_of_agents)>sparseness,1,0)
    weight_matrix = np.random.rand(number_of_agents,number_of_agents)
    a=relationship_matrix.copy()
    consensus = pd.DataFrame({'t0':beliefs})
    b=beliefs
    for i in range(time_periods):
        relationship_matrix=neighbour_change(number_of_agents,relationship_matrix,b,0.1,0.1)
        b=next_period_beliefs(number_of_agents,b, relationship_matrix, weight_matrix)
        consensus.insert(loc= len(consensus.columns),column = ('t'+str(i+1)),value= b)
    print((number_of_agents*number_of_agents)-(a==relationship_matrix).sum())
    return consensus

In [ ]:
def auto_siml(number_of_agents):
    #Network = nx.Graph()
    # randomly generating initial belief values i.e. x(t) for the agents and storing them in the node 
    sparseness=0.75
    beliefs=np.random.uniform(0,1,number_of_agents)
    a = np.random.rand(number_of_agents,number_of_agents)
    relationship_matrix = np.where((np.tril(a) + np.tril(a, -1).T)+np.eye(number_of_agents)>sparseness,1,0)
    weight_matrix = np.random.rand(number_of_agents,number_of_agents)
    a=relationship_matrix.copy()
    consensus = pd.DataFrame({'t0':beliefs})
    b=beliefs
    t=0
    consensus_final = list(consensus[consensus.columns[-1]])
    dif_sum=[]
    for j in consensus_final:
        for k in consensus_final:
            dif_sum.append(abs(j-k))
    while(sum(dif_sum)/len(dif_sum) > 0.001):
        t=t+1
        if t > 99:
            break
        relationship_matrix=neighbour_change(number_of_agents,relationship_matrix,b,0.1,0.1)
        b=next_period_beliefs(number_of_agents,b, relationship_matrix, weight_matrix)
        consensus.insert(loc= len(consensus.columns),column = ('t'+str(t)),value= b)
        consensus_final = list(consensus[consensus.columns[-1]])
        dif_sum=[]
        for j in consensus_final:
            for k in consensus_final:
                dif_sum.append(abs(j-k))
    return t

In [ ]:
restricted_graph_w_nei_sim = pd.DataFrame()
for s in tqdm(range(101)):
    #print(s)
    time_required=[]
    for i in range(2,102):
        #print(i)
        time_required.append(auto_siml(i))     
    restricted_graph_w_nei_sim.insert(loc= len(restricted_graph_w_nei_sim.columns),column = "sim_"+str(s),value = time_required)

In [ ]:
mean=restricted_graph_w_nei_sim.mean(axis=1)
restricted_graph_w_nei_sim.insert(loc=len(restricted_graph_w_nei_sim.columns),column = "number_of_agents",value = np.linspace(2,101,100) )
restricted_graph_w_nei_sim.insert(loc=len(restricted_graph_w_nei_sim.columns),column = "Mean",value =mean )
restricted_graph_w_nei_sim.to_csv('restricted_graph_w_nei_S=0.75.csv')

#Restricted Graph Model with Unlimited Neighbour Change 

In [ ]:
def siml(number_of_agents,time_periods):
    #Network = nx.Graph()
    # randomly generating initial belief values i.e. x(t) for the agents and storing them in the node 
    sparseness=0.5
    beliefs=np.random.uniform(0,1,number_of_agents)
    a = np.random.rand(number_of_agents,number_of_agents)
    relationship_matrix = np.where((np.tril(a) + np.tril(a, -1).T)+np.eye(number_of_agents)>sparseness,1,0)
    weight_matrix = np.random.rand(number_of_agents,number_of_agents)
    a=relationship_matrix.copy()
    consensus = pd.DataFrame({'t0':beliefs})
    b=beliefs
    for i in range(time_periods):
        saved_relationmatrix = relationship_matrix.copy()
        last_change=0
        relationship_matrix=neighbour_change(number_of_agents,relationship_matrix,b,0.1,0.1)
        change = (relationship_matrix==relationship_matrix.T).sum() - (saved_relationmatrix==relationship_matrix).sum()
        while (change-last_change >0):
            last_change = change
            relationship_matrix=neighbour_change(number_of_agents,relationship_matrix,b,0.1,0.1)
            change = (relationship_matrix==relationship_matrix.T).sum() - (saved_relationmatrix==relationship_matrix).sum()
        b=next_period_beliefs(number_of_agents,b, relationship_matrix, weight_matrix)
        consensus.insert(loc= len(consensus.columns),column = ('t'+str(i+1)),value= b)
    print((number_of_agents*number_of_agents)-(a==relationship_matrix).sum())
    return consensus

In [ ]:
def auto_siml(number_of_agents):
    #Network = nx.Graph()
    # randomly generating initial belief values i.e. x(t) for the agents and storing them in the node 
    sparseness=0.75
    beliefs=np.random.uniform(0,1,number_of_agents)
    a = np.random.rand(number_of_agents,number_of_agents)
    relationship_matrix = np.where((np.tril(a) + np.tril(a, -1).T)+np.eye(number_of_agents)>sparseness,1,0)
    weight_matrix = np.random.rand(number_of_agents,number_of_agents)
    a=relationship_matrix.copy()
    consensus = pd.DataFrame({'t0':beliefs})
    b=beliefs
    t=0
    consensus_final = list(consensus[consensus.columns[-1]])
    dif_sum=[]
    for j in consensus_final:
        for k in consensus_final:
            dif_sum.append(abs(j-k))
    while(sum(dif_sum)/len(dif_sum) > 0.001):
        t=t+1
        if t > 199:
            break
        saved_relationmatrix = relationship_matrix.copy()
        last_change=0
        relationship_matrix=neighbour_change(number_of_agents,relationship_matrix,b,0.1,0.1)
        change = (relationship_matrix==relationship_matrix.T).sum() - (saved_relationmatrix==relationship_matrix).sum()
        while (change-last_change >0):
            last_change = change
            relationship_matrix=neighbour_change(number_of_agents,relationship_matrix,b,0.1,0.1)
            change = (relationship_matrix==relationship_matrix.T).sum() - (saved_relationmatrix==relationship_matrix).sum()
        b=next_period_beliefs(number_of_agents,b, relationship_matrix, weight_matrix)
        consensus.insert(loc= len(consensus.columns),column = ('t'+str(t)),value= b)
        consensus_final = list(consensus[consensus.columns[-1]])
        dif_sum=[]
        for j in consensus_final:
            for k in consensus_final:
                dif_sum.append(abs(j-k))
    return t

In [ ]:
restricted_graph_w_uc_nei_sim = pd.DataFrame()
for s in tqdm(range(101)):
    #print(s)
    time_required=[]
    for i in range(2,102):
        print(i)
        time_required.append(auto_siml(i))     
    restricted_graph_w_uc_nei_sim.insert(loc= len(restricted_graph_w_uc_nei_sim.columns),column = "sim_"+str(s),value = time_required)

In [ ]:
mean=restricted_graph_w_uc_nei_sim.mean(axis=1)
restricted_graph_w_uc_nei_sim.insert(loc=len(restricted_graph_w_uc_nei_sim.columns),column = "number_of_agents",value = np.linspace(2,101,100) )
restricted_graph_w_uc_nei_sim.insert(loc=len(restricted_graph_w_uc_nei_sim.columns),column = "Mean",value =mean )
restricted_graph_w_uc_nei_sim.to_csv('restricted_graph_w_uc_nei_sim_S=0.75_test.csv')